# 2、torch.utils.data.DataLoader。

如果说` Dataset `是一个`“数据仓库”`，定义了数据的总量和获取单个数据的方法，那么` DataLoader `就是一个高效的`“数据搬运工”`，负责从仓库中取出数据，经过智能打包和运输，高效地送达“模型”这个工厂。

---

## ⚠️ 核心概念：DataLoader 是什么？
`torch.utils.data.DataLoader` 是一个 Python 可`迭代对象` (iterable)。它将一个 Dataset 对象包装起来，为我们提供了一种简单、高效、可定制的方式来迭代访问数据集。

它解决了` Dataset `无法直接处理的几个关键问题，使得大规模数据训练成为可能：

- 1.`批量处理 (Batching)`：
模型训练通常采用小批量随机梯度下降 (mini-batch SGD)，一次处理一小批数据而不是单个样本。`DataLoader` 自动将从` Dataset`中取出的单个样本打包成一个批次 (batch)。

- 2.`数据打乱 (Shuffling)`：
为了让模型有更好的泛化能力，避免过拟合，我们需要在每个`训练周期` (epoch) 开始时打乱数据顺序。`DataLoader `可以通过一个简单的参数` shuffle=True `实现这一点。

- 3.`并行加载 (Parallel Loading)`：
数据加载（从硬盘读取、预处理）通常是 CPU 密集型任务。如果串行加载，GPU 可能会花费大量时间等待 CPU 准备好数据，造成“算力饥饿”。`DataLoader `可以使用多个子进程 (num_workers) 在后台并行加载数据，让数据准备和模型计算同时进行，极大地提升了训练效率。

- 4.`数据整合 (Collation)`：
将多个独立的样本组合成一个批次张量 (batch tensor) 的过程。`DataLoader `有默认的整合逻辑，也支持用户自定义。

---

## ⚠️ 核心参数详解
`DataLoader` 的构造函数有很多参数，我们来讲解其中最重要、最常用的几个：

In [ ]:
torch.utils.data.DataLoader(
    dataset,            # 数据来源 (Dataset对象)，必须实现 __len__() 和 __getitem__()
    batch_size=1,       # 每个批次包含的样本数量
    shuffle=False,      # 是否打乱顺序 (训练时通常设为True，验证/测试时设为False)
    sampler=None,       # 自定义采样器对象，用于控制数据加载顺序 (与shuffle互斥)
    batch_sampler=None, # 类似sampler但每次返回一个batch的索引 (与batch_size/shuffle/sampler/drop_last互斥)
    num_workers=0,      # 用于数据加载的子进程数量 (0表示在主进程加载)
    collate_fn=None,    # 合并样本列表形成batch的函数 (默认是torch.stack)
    pin_memory=False,             # 若为True，将数据加载到CUDA固定内存(可加速GPU传输)
    drop_last=False,              # 是否丢弃最后一个不完整的batch (当样本数不能被batch_size整除时)
    timeout=0,                    # 数据加载的超时时间(秒)，0表示不超时
    worker_init_fn=None,          # 每个worker初始化函数
    multiprocessing_context=None, # 多进程上下文
    generator=None,               # 用于生成随机数的生成器对象
    prefetch_factor=2,            # 每个worker预加载的batch数量
    persistent_workers=False      # 是否保持workers存活(避免每个epoch重建)
)

## 参数详解
---
#### `dataset`（必需）
- **类型**：`torch.utils.data.Dataset` 对象  
- **作用**：指定`数据来源`，是 `DataLoader` 的数据源。必须传入一个继承自 `Dataset` 的实例。

---

#### `batch_size`
- **类型**：`int`，默认为 `1`  
- **作用**：定义每个批次包含的样本数量。  
- **建议**：根据 GPU 显存和模型复杂度调整，常见值为 `16`, `32`, `64`, `128`。

---
#### `shuffle`
- **类型**：`bool`，默认为 `False`  
- **作用**：是否在每个 epoch 开始时打乱数据顺序。  
- **建议**：
  - ✅ 训练时：`True`（提升模型泛化能力）
  - ❌ 验证/测试时：`False`（保证评估一致性）

---

#### `num_workers`
- **类型**：`int`，默认为 `0`  
- **作用**：用于数据加载的子进程数量。
  - `0`：所有数据在主进程中加载（同步）。
  - `> 0`：使用多个子进程异步加载数据，提升速度。
- **建议**：
  - 一般设为 `4`, `8`, `16`，建议不超过 CPU 核心数。
  - Windows 上注意避免 `num_workers > 0` 导致的 `freeze_support` 问题（建议在 `if __name__ == '__main__':` 中运行）。

---

#### `pin_memory`
- **类型**：`bool`，默认为 `False`  
- **作用**：若为 `True`，将数据加载到“固定内存”（pinned memory），加快从 CPU 到 GPU 的传输速度。  
- **建议**：
  - ✅ 使用 GPU 训练时：强烈建议设为 `True`
  - ❌ CPU 训练时：无需开启

---

#### `drop_last`
- **类型**：`bool`，默认为 `False`  
- **作用**：当样本总数不能被 `batch_size` 整除时，最后一个批次样本数会不足。若设为 `True`，则丢弃这个不完整的批次。  
- **应用场景**：
  - 某些模型要求输入尺寸固定（如部分 RNN、GAN）
  - 批归一化（BatchNorm）在小批次上不稳定时

---

#### `collate_fn`
- **类型**：可调用函数（`callable`），默认为 `None`  
- **作用**：自定义如何将多个样本组合成一个批次。默认函数会将张量堆叠（`torch.stack`）。  
- **默认行为**：
  ```python
  # 默认 collate_fn 会做类似操作
  batch = {
      'images': torch.stack([s['image'] for s in samples]),
      'labels': torch.tensor([s['label'] for s in samples])
  }

---

### ✅ 使用建议总结

| 参数 | 训练模式建议 | 验证/测试建议 |
|------|---------------|----------------|
| `shuffle` | `True` | `False` |
| `num_workers` | `4~16`（根据 CPU） | `4~8` |
| `pin_memory` | `True`（GPU） | `True`（GPU） |
| `drop_last` | `True`（若模型敏感） | `False` |
| `collate_fn` | 按需自定义 | 按需自定义 |

---

## 🔄 DataLoader 工作流程详解
当你开始在一个` DataLoader `上进行迭代时（例如：for batch in data_loader:），其内部会自动执行一系列高效的操作，实现数据加载与模型训练的流水线并行。整个流程如下：

### 1️⃣ 生成索引（Index Generation）
`DataLoader` 首先通过一个 `Sampler`（采样器） 生成当前批次所需的样本索引列表。\
根据 `shuffle` 参数选择不同的采样策略：
- `✅ shuffle=True → 使用 RandomSampler（随机打乱顺序）`
- `❌ shuffle=False → 使用 SequentialSampler（按顺序采样）`
这些索引决定了本次需要加载哪些样本。

---

### 2️⃣ 分发任务（Task Distribution）
如果设置了` num_workers `> 0，`DataLoader `会将这批索引分发给`多个子进程`（worker processes）。
每个子进程负责加载一部分数据，实现任务并行化。

---

### 3️⃣ 并行加载（Parallel Data Loading）
每个子进程独立执行：

dataset[index]

- `即调用 Dataset 的 __getitem__ 方法。`

- 加载过程包括：
    - `文件读取（如图像、文本）`
    - `数据解码（如 PIL 加载图片）`
    - `应用 transform 进行预处理`
    - `所有子进程并行运行，显著提升 I/O 效率，避免成为训练瓶颈。`

---

### 4️⃣ 整合数据（Collation）
主进程收集所有子进程返回的单个样本，组成一个列表：

batch_list = [sample_1, sample_2, ..., sample_batch_size]

然后调用 `collate_fn` 函数，将该列表整合为一个完整的批次：\
🔹 默认行为：使用 `torch.stack()` 将张量堆叠成一个大张量。\
🔹 自定义需求：对于变长数据（如` NLP `句子），需自定义` collate_fn `实现 `padding` 或 `packing`。\
输出通常为 `(inputs, labels)` 元组或`字典形式`。

---

### 5️⃣ 返回批次（Yield Batch）
整合后的批次数据通过` yield `返回给训练循环。\
⚡ 关键优势：在主进程进行前向传播、反向传播等计算的同时，子进程已在后台加载下一个批次的数据，形成“流水线并行（Pipelining）”，最大化 GPU 利用率。

---

### 代码示例
我们继续使用之前定义的`CatsAndDogsDataset`。

In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader

# 假设 CatsAndDogsDataset 类已经在这里定义好了
# class CatsAndDogsDataset(Dataset):
#     ...

# 1. 定义数据变换
data_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 2. 实例化 Dataset
dataset_path = 'data/'
train_dataset = CatsAndDogsDataset(root_dir=dataset_path, transform=data_transform)

# 3. 实例化 DataLoader，并配置核心参数
# 这是训练集加载器，所以 shuffle=True
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=64,       # 每个批次加载 64 张图片
    shuffle=True,        # 每个 epoch 都打乱数据
    num_workers=4,       # 使用 4 个子进程来加载数据
    pin_memory=True      # 如果使用 GPU，设置为 True
)

# 4. 在训练循环中使用 DataLoader
print(f"开始遍历 train_loader...")
# DataLoader 是一个迭代器，我们可以像遍历列表一样遍历它
for i, (images, labels) in enumerate(train_loader):
    # 将数据移动到 GPU (如果可用)
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # images = images.to(device)
    # labels = labels.to(device)
    
    # 打印批次数据的形状
    print(f"批次 {i+1}:")
    print(f"  - 图像批次的形状: {images.shape}")  # torch.Size([64, 3, 224, 224])
    print(f"  - 标签批次的形状: {labels.shape}")  # torch.Size([64])

    # 在这里，可以将 images 和 labels 送入模型进行训练
    # e.g., outputs = model(images)
    #        loss = criterion(outputs, labels)
    #        ...

    # 为了演示，我们只遍历几个批次就退出
    if i >= 2:
        break